In [39]:
import numpy as np
import pandas as pd
import researchpy as rp
import scipy.stats as stats
from scipy.stats import shapiro
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

In [1157]:
df = pd.read_excel('../Figures and tables/Candidate genes.xlsx')

In [1158]:
df2 = pd.read_csv('../Figures and tables/Expression dendogram 10 clusters.csv')

In [1159]:
df2 = pd.merge(df,df2, how = 'right').fillna('No')

In [1160]:
df3 = pd.read_table('../../Data/MSU7/all.locus_brief_info.7.0').drop_duplicates(['locus', 'annotation'])[['locus', 'annotation']]

In [1161]:
df3 = pd.merge(df2, df3, left_on = 'Locus ID', right_on = 'locus')

In [1068]:
df3[['Locus ID', 'QTL', 'annotation', 'Functionally annotated', 'Outlier in expression', 'Expression cluster',
    'Special interest']].to_csv('../Figures and tables/All candidate genes.csv', index = False)

In [1070]:
for sr in SeqIO.parse('../../Data/MSU7/all.seq', 'fasta'):
    if sr.id in df3['Locus ID'].to_list():
        with open('../Sequence and SNP data/340 candidate genes.fasta', 'a') as f:
            f.write('>%s\n%s\n' %(sr.description, sr.seq))

In [1072]:
c = 0
for sr in SeqIO.parse('../Sequence and SNP data/340 candidate genes.fasta', 'fasta'):
    c+=1
print(c)

340


In [ ]:
#gonna go over and run blast

In [1162]:
df4 = pd.read_table('../Sequence and SNP data/Bout340can.tsv', header = None)
df4['Start'] = df4[[8,9]].min(axis = 1)
df4['End'] = df4[[8,9]].max(axis = 1)
t = []
for s,e in df4[[8,9]].itertuples(index = False):
    if s < e:
        t.append('forward')
    else:
        t.append('reverse')
df4['Strand'] = t

In [1163]:
df4[df4[0].duplicated(keep = False)]

,0,1,2,3,4,5,6,7,8,9,10,11,Start,End,Strand
47,LOC_Os02g15178,chr02,100.000,2139,0,0,1,2139,8460664,8458526,0.000000e+00,3951,8458526,8460664,reverse
48,LOC_Os02g15178,chr02,100.000,2139,0,0,1,2139,8465035,8467173,0.000000e+00,3951,8465035,8467173,forward
122,LOC_Os05g30410,chr05,100.000,333,0,0,1,333,17627376,17627044,4.060000e-175,616,17627044,17627376,reverse
123,LOC_Os05g30410,chr04,93.994,333,20,0,1,333,33855052,33854720,9.160000e-142,505,33854720,33855052,reverse
144,LOC_Os06g35574,chr06,100.000,898,0,0,1,898,20761602,20760705,0.000000e+00,1659,20760705,20761602,reverse
145,LOC_Os06g35574,chr03,85.761,927,88,31,1,898,31959677,31960588,0.000000e+00,941,31959677,31960588,forward
206,LOC_Os09g15350,chr09,100.000,2370,0,0,1,2370,9385267,9387636,0.000000e+00,4377,9385267,9387636,forward
207,LOC_Os09g15350,chr05,98.819,2370,13,2,1,2370,17778572,17776218,0.000000e+00,4207,17776218,17778572,reverse
318,LOC_Os12g28137,chr12,100.000,1987,0,0,1,1987,16616739,16618725,0.000000e+00,3670,16616739,16618725,forward
319,LOC_Os12g28137,chr12,100.000,1987,0,0,1,1987,16640708,16642694,0.000000e+00,3670,16640708,16642694,forward


In [1164]:
#fix nonlocus hits
df5 = df4.loc[[123, 145, 207],:].copy()
df5.to_csv('../Figures and tables/Nonloci Bhits.csv', index = False)

In [1165]:
df4 = df4.drop(index = [123,145,207])

In [1166]:
df4 = pd.merge(df4, df3[['Locus ID', 'QTL']], left_on = 0, right_on = 'Locus ID')[[0, 1, 'Start', 'End', 'Strand', 'QTL']]

In [1167]:
df5 = pd.concat([df4.groupby('QTL').min()['Start'], df4.groupby('QTL').max()['End']], axis = 1)

In [1168]:
df5 = pd.merge(df5.reset_index(), df4[['QTL', 1]].drop_duplicates())

In [1115]:
df5.to_csv('../Sequence and SNP data/340can range.csv', index = False)

In [ ]:
#Filter master VCF file

In [1169]:
df5 = pd.read_csv('../../WFC2.vcf', sep = '\t')[['#CHROM', 'POS', 'ID', 'REF', 'ALT']]

In [1170]:
t = []
for c, p in df5[['#CHROM', 'POS']].itertuples(index = False):
    f = False
    for l,o,s,e in df4[df4[1] == c][[0, 'Strand', 'Start', 'End']].itertuples(index = False):
        if s <= p <= e:
            t.append((l, o, s, e))
            f = True
            break
    if f == False:
        t.append(('None', o, s, e))

In [1171]:
df5['Gene'] = list(zip(*t))[0]
df5['Strand'] = list(zip(*t))[1]
df5['Start'] = list(zip(*t))[2]
df5['End'] = list(zip(*t))[3]

In [1172]:
df5 = df5[df5['Gene'] != 'None']

In [187]:
#don't run
ds = []
for sr in SeqIO.parse('../../Data/RAP-DB/IRGSP-1.0_genome.fasta', 'fasta'):
    tdf = df5[df5['#CHROM'] == sr.id]
    tdf = tdf.drop_duplicates(['Gene', 'Start', 'End'])
    for r,p,g,s,s2,e in tdf[['REF', 'POS', 'Gene', 'Strand', 'Start', 'End']].itertuples(index = False):
        assert sr.seq[p-1].lower() == r[0].lower()        
        if s == 'forward':
            ds.append(SeqRecord(sr.seq[(s2-1):e], id = g + '_' + str(s2)))
        else:
            ds.append(SeqRecord(sr.seq[(s2-1):e].reverse_complement(), id = g + '_' + str(s2)))
with open('extracted_candidates.fasta', 'w') as f:
    SeqIO.write(ds, f, 'fasta')

In [1134]:
c = 0
for sr in SeqIO.parse('../Sequence and SNP data/340 candidate genes.fasta', 'fasta'):
    if sr.id not in df4[0].to_list():
        print(sr.id)
    c+=1
print(c)

340


In [1249]:
df6 = pd.read_table('../../Data/MSU7/all.locus_brief_info.7.0')

In [1250]:
df6 = df6[df6['is_expressed'] == 'Y']

In [1186]:
df6r = df6[df6['is_representative'] == 'Y']

In [1188]:
df6nr = df6[df6['is_representative'] == 'N']

In [1189]:
for l in df6nr['locus'][df6nr['locus'].duplicated()].to_list():
    if l in df4[0].to_list():
        print(l)

LOC_Os03g03470
LOC_Os03g03470
LOC_Os03g03610
LOC_Os03g29350
LOC_Os05g30280
LOC_Os05g30400
LOC_Os05g30400
LOC_Os07g36600
LOC_Os07g37580
LOC_Os08g43360
LOC_Os09g15430
LOC_Os09g15430
LOC_Os09g34280
LOC_Os09g34280
LOC_Os09g34310
LOC_Os09g34850
LOC_Os10g35030
LOC_Os10g35070
LOC_Os10g35110
LOC_Os12g41650
LOC_Os12g41650
LOC_Os12g41710
LOC_Os12g41710


In [1292]:
df7 = pd.merge(df4,df6[['locus', 'model']], left_on = 0, right_on = 'locus')

In [1293]:
df7 = df7.drop_duplicates(['locus', 'model'])

In [1227]:
t = []
for sr in SeqIO.parse('../../Data/MSU7/all.cds', 'fasta'):
    t.append(sr.id)
for m in df7['model'].to_list():
    if m not in t:
        print(m)

In [1236]:
t = []
for sr in SeqIO.parse('../../Data/MSU7/all.cds', 'fasta'):
    t.append((sr.id, sr.description, sr.seq))

In [1237]:
with open('../Sequence and SNP data/427 candidate cds.fasta', 'a') as f:
    for r in t:
        if r[0] in df7['model'].to_list():
            f.write('>%s\n%s\n' %(r[1], r[2]))

In [1238]:
c = 0
for sr in SeqIO.parse('../Sequence and SNP data/427 candidate cds.fasta', 'fasta'):
    c+=1
print(c)

427


In [ ]:
#take a trip to beautiful intron extractor

In [1294]:
t3 = []
for sr in SeqIO.parse('../Sequence and SNP data/427extracted_genes_noint.fasta', 'fasta'):
    t3.append((sr.id, sr.seq))

In [1295]:
df8 = pd.DataFrame(t3).drop_duplicates(1)

In [1243]:
for i,s in df7.itertuples(index = False):
    with open('../Sequence and SNP data/392 unique candidates.fasta', 'a') as f:
        f.write('>%s\n%s\n' % (i, str(s)))

In [1244]:
c = 0
for sr in SeqIO.parse('../Sequence and SNP data/392 unique candidates.fasta', 'fasta'): 
    c+=1
print(c)

392


In [1300]:
df7 = pd.merge(df5, df7[['QTL', 'locus', 'model']], left_on = 'Gene', right_on = 'locus').drop(columns = ['locus'])

In [1302]:
df7['Gene POS'] = df7['POS'] - df7['Start']

In [1303]:
t = []
for sr in SeqIO.parse('../Sequence and SNP data/392 unique candidates.fasta', 'fasta'):
    t.append((sr.id, sr.seq))
df8 = pd.DataFrame(t, columns = ['Hit key', 'Gene Seq'])

In [1304]:
df7 = df7.rename(columns = {'model' : 'Hit key'})

In [1307]:
df8 = pd.merge(df7,df8)

In [1309]:
t = [] 
for s, sq in df8[['Strand', 'Gene Seq']].itertuples(index = False):
    if s == 'reverse':
        t.append(sq.reverse_complement())
    else:
        t.append(sq)
df8['5to3 seq'] = t

In [1327]:
t = []
for r,s,hk,snp,sqf,sqr in df8[['REF', 'Strand', 'Hit key', 'Gene POS', 'Gene Seq', '5to3 seq']].itertuples(index = False): 
    if s == 'forward':
        if not (sqf[snp:(snp + len(r))].lower() == r.lower() or '-' in sqf[snp:(snp + len(r))].lower()):
            print(sqf[snp:(snp + len(r))].lower(), r.lower(), hk)
            t.append(hk)
    else:
        if not (sqr[snp:(snp + len(r))].lower() == r.lower() or '-' in sqr[snp:(snp + len(r))].lower()):
            print(sqr[snp:(snp + len(r))].lower(), r.lower(), hk)
            t.append(hk)

g c LOC_Os01g66030.2
c a LOC_Os02g14860.1
c g LOC_Os02g14860.1
a c LOC_Os07g36600.1
t g LOC_Os07g36600.1
t g LOC_Os07g36600.1
a g LOC_Os07g36600.1
t c LOC_Os07g36600.1
a c LOC_Os07g36600.1
a c LOC_Os07g36600.1
a g LOC_Os09g15480.2
ctaa ctaat LOC_Os09g15500.1
atct ggct LOC_Os12g27994.1
ctgc ctgt LOC_Os12g27994.1
c g LOC_Os12g27994.1
c g LOC_Os12g27994.1
t a LOC_Os12g27994.1
c g LOC_Os12g27994.1
g a LOC_Os12g27994.1
c t LOC_Os12g27994.1
c g LOC_Os12g27994.1
c a LOC_Os12g27994.1
gcctggg gagcttc LOC_Os12g27994.1
c t LOC_Os12g41460.1
c a LOC_Os12g41460.1
c t LOC_Os12g41460.1
g c LOC_Os12g41460.1
g a LOC_Os12g41460.1
c g LOC_Os12g41460.1
g c LOC_Os12g41460.1
g t LOC_Os12g41460.1
g t LOC_Os12g41460.1
c g LOC_Os12g41460.1
g c LOC_Os12g41460.1
c g LOC_Os12g41460.1
c g LOC_Os12g41460.1
g c LOC_Os12g41460.1
t g LOC_Os12g41460.1
t c LOC_Os12g41460.1
c t LOC_Os12g41460.1
c t LOC_Os12g41460.1
g c LOC_Os12g41460.1
g a LOC_Os12g41460.1
g t LOC_Os12g41460.1
t c LOC_Os12g41460.1
g c LOC_Os12g41460.1
g t

In [1329]:
#remove faulty alignments
t2 = []
for a in set(t):
    t2 = t2 + list(df8[df8['Hit key'] == a].index)

In [1337]:
df9 = df8.drop(index = t2)

In [1331]:
t = []
for r,s,hk,snp,sqf,sqr in df9[['REF', 'Strand', 'Hit key', 'Gene POS', 'Gene Seq', '5to3 seq']].itertuples(index = False): 
    if s == 'forward':
        if not (sqf[snp:(snp + len(r))].lower() == r.lower() or sqf[snp].lower() == '-'):
            print(sqf[snp:(snp + len(r))].lower(), r.lower(), hk)
            t.append(hk)
    else:
        if not (sqr[snp:(snp + len(r))].lower() == r.lower() or sqr[snp].lower() == '-'):
            print(sqr[snp:(snp + len(r))].lower(), r.lower(), hk)
            t.append(hk)

gg--- ggggt LOC_Os01g66310.1
tcagtatttgta------ tcagtatttgtactggag LOC_Os03g03520.1
t--- tggc LOC_Os05g30454.1
at- atc LOC_Os12g41660.1
gg-- ggtc LOC_Os12g41720.1


In [395]:
#not needed right now
#correcting reference substitutions
t = []
for r,s,hk,snp,sqf,sqr in df9[['REF', 'Strand', 'Hit key', 'Gene POS', 'Gene Seq', '5to3 seq']].itertuples(index = False): 
    if s == 'forward':
        if not (sqf[snp].lower() == r.lower() or sqf[snp].lower() == '-'):
            if (len(r) != 1):
                if not (sqf[snp:snp+len(r)].lower() == r.lower() or sqf[snp].lower() == '-'):
                    tsq = str(sqf[:snp]) + r + str(sqf[snp + len(r):])
                    t.append(Seq(tsq, generic_dna))
                else:
                    t.append(Seq(str(sqf), generic_dna))
            else:
                tsq = str(sqf[:snp]) + r + str(sqf[snp + len(r):])
                t.append(Seq(tsq, generic_dna))        
        else:
            t.append(Seq(str(sqf), generic_dna))
    else:
        if not (sqr[snp].lower() == r.lower() or sqr[snp].lower() == '-'):
            if (len(r) != 1):
                if not (sqr[snp:snp+len(r)].lower() == r.lower() or sqr[snp].lower() == '-'):
                    tsq = str(sqr[:snp]) + r + str(sqr[snp + len(r):])
                    t.append(Seq(tsq, generic_dna))
                else:
                    t.append(Seq(str(sqr), generic_dna))
            else:
                tsq = str(sqr[:snp]) + r + str(sqr[snp + len(r):])
                t.append(Seq(tsq, generic_dna))
        else:
            t.append(Seq(str(sqr), generic_dna))

In [396]:
#swapping with rename
#df9['Extract'] = t

In [1338]:
df9 = df9.rename(columns = {'5to3 seq' : 'Extract'}).drop(columns = ['Gene Seq'])

In [1340]:
for r, snp, sq in df9[['REF', 'Gene POS', 'Extract']].itertuples(index = False): 
    if not (sq[snp].lower() == r.lower() or sq[snp].lower() == '-'):
        if (len(r) != 1):
            if not (sq[snp:snp+len(r)].lower() == r.lower() or sq[snp].lower() == '-'):
                print(sq[snp:snp+len(r)].lower(), r.lower())

gg--- ggggt
tcagtatttgta------ tcagtatttgtactggag
t--- tggc
at- atc
gg-- ggtc


In [1342]:
t = []
for i,r in df9.iterrows():
    if ',' in r['ALT']:
        for a in r['ALT'].split(','):
            r2 = r.copy()
            r2.loc['ALT'] = a
            t.append(r2)
    else:
        t.append(r)

In [1343]:
df10 = pd.DataFrame(t)

In [1345]:
#mutagenesis
t = []
for r, a, p, esq in df10[['REF', 'ALT', 'Gene POS', 'Extract']].itertuples(index = False):
    if esq[p] == '-':
        t.append(Seq(str(esq), generic_dna))
    else:
        tsq = str(esq)
        tsq = tsq[:p] + a + tsq[p+len(r):]
        t.append(Seq(tsq, generic_dna))

In [1346]:
df10['Mutated'] = t

In [1347]:
t = []
for esq, msq in df10[['Extract', 'Mutated']].itertuples(index = False):
    if len(esq) != len(msq) and (len(msq) - len(esq)) % 3 != 0:
        t.append('yes')
    else:
        t.append('no')    

In [1348]:
df10['Frameshift'] = t

In [1349]:
for i,r in df10[df10['Frameshift'] == 'yes'].iterrows():
    if len(r['REF']) != 1 or len(r['ALT']) != 1:
        if (len(r['ALT']) - len(r['REF'])) % 3 == 0:
            print(r['REF'], r['ALT'])
        if r['Frameshift'] == 'no':
            print(r['REF', 'ALT'])

In [1350]:
t = []
for p, r, a, s, e, m in df10[['Gene POS', 'REF', 'ALT', 'Strand', 'Extract', 'Mutated']].itertuples(index = False):
    
#     if len(r) < len(a):
#         t.append('insertion')
#         continue
#     elif len(r) > len(a):
#         t.append('deletion')
#         continue
    
    if s == 'forward':
    
        op = Seq(str(e).replace('-', ''), generic_dna).translate()
        np = Seq(str(m).replace('-', ''), generic_dna).translate()
        if op == np:
            t.append('synonymous')
        else:
            
            no_nonsense_switch = 'on'
            for i in range(min(len(op), len(np))):
                if np[i] != op[i]:
                    if np[i] == '*':
                        t.append('nonsense')
                        no_nonsense_switch = 'off'
                        break
            if no_nonsense_switch == 'on':
                t.append('missense')
            
            
    else:
        op = Seq(str(e).replace('-', ''), generic_dna).reverse_complement().translate()
        np = Seq(str(m).replace('-', ''), generic_dna).reverse_complement().translate()
        if op == np:
            t.append('synonymous')
        else:
            
            no_nonsense_switch = 'on'
            for i in range(min(len(op), len(np))):
                if np[i] != op[i]:
                    if np[i] == '*':
                        t.append('nonsense')
                        no_nonsense_switch = 'off'                        
                        break 
            if no_nonsense_switch == 'on':
                t.append('missense')


c:\users\michael\appdata\local\programs\python\python38\lib\site-packages\Bio\Seq.py:2738: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


In [1351]:
df10['Effect'] = t

In [1352]:
t = []
for p, e in df10[['Gene POS', 'Extract']].itertuples(index = False):
    if e[p] == '-':
        t.append('intron')
    else:
        t.append('exon')

In [1353]:
df10['Site'] = t

In [1354]:
df10 = df10.reset_index(drop = True)

In [1356]:
df10['MID'] = df10.index

In [1359]:
df10.drop(columns = ['Extract', 'Mutated']).to_clipboard(index = False)
#run testcases before saving from clipboard
#rename hit key to transcript

In [1382]:
str(Seq(str(df10.loc[161, 'Extract']).replace('-', ''), generic_dna).translate())

'MGEFSGMATTGALYSGQVMPITSRDGSGTKPKGTRLERAIRDLQKIAAEYRPPAIDINEVDPNGQVAVKRRLPPEVKQKLAKVARLSANHGKIQEHELMDRLMGIVGHLVQRRTLRRNMKEMVESGLSAKQEKADKFQRVKMEINEMIKSRVAAKAKVNEHHSGSADDFQIANDEKRYLKGKSVMDAALEDRICDLYDLYVEGMDEDKGPQSRKLYVELAELWPEGSMDNVGIKDAINRSKERRRSLYNQQKVRNEERMKRKRLAAAAKLQDGYPVVMQSALIQQVAQPPITNPVATYPVTDQGSKSFDRVREISASANPDDINRNTGEMKKKKRKPESDLVDTQANAMKGPSQHVEKNKPPKRADEAVETVLCLPFYDQQPS*'

In [1383]:
str(Seq(str(df10.loc[161, 'Mutated']).replace('-', ''), generic_dna).translate())

'MGEFSGMATTGALYSGQVMPITSRDGSGTKPKGTRLERAIRDLQKIAAEYRPPAIDINEVDPNGQVAVKRRLPPEVKQKLAKVARLSANHGKIQEHELMDRLMGIVGHLVQRRTLRRNMKEMVESGLSAKQEKADKFQRVKMEINEMIKSRVAAKAKVNEHHSGSADDFQIANDEKRYLKGKSVMDAALEDRICDLYDLYVEGMDEDKGPQSRKLYVELAELWPEGSMDNVGIKDAINRSKERRRSLYNQQKVRNEERMKRKRLAAAAKLQDGYPVVMQSALIQQVAQPPITNPVTDQGSKSFDRVREISASANPDDINRNTGEMKKKKRKPESDLVDTQANAMKGPSQHVEKNKPPKRADEAVETVLCLPFYDQQPS*'

In [1384]:
str(Seq(str(df10.loc[1033, 'Mutated']).replace('-', ''), generic_dna).reverse_complement().translate())

'MEKEVEEEEEEEEAEASRKKGERGLGEGECGAPSSKKRDPELELRMKQKAAEWHRKAREETLKEIAKEMAKYPNEDWSDTPGVKAREYREDWEYRWSAIFGPYDTISPIPPMRYTHRKDDSMPRHISVRHTLQIISVKIKGIRGGLQWPINVFGLIAARDTIDRNRIMIFNRTRDNCQTITKEDR*LLLTGPTRAVVVSDPVYFEAPLKVKGSVESEDKDLSFLAVPLTGASDRGETRLVNREYTSRLSTLELTFGFVVESLEASISVRIIDGSWKDGFRGAFTAHTPSLKDNKVLLLDSGYCEMVPVTADRMIKLSRHVVSVEGEGDLTVSVLALGTDNVIEDEKDFTPKEAGMSQSSLDVGFCKLEVTVNWSLLSLLPDGYT*'

In [1385]:
str(Seq(str(df10.loc[1033, 'Extract']).replace('-', ''), generic_dna).reverse_complement().translate())

'MEKEVEEEEEEEEAEASRKKGERGLGEGECGAPSSKKRDPELELRMKQKAAEWHRKAREETLKEIAKEMAKYPNEDWSDTPGVKAREYREDWEYRWSAIFGPYDTISPIPPMRYTHRKDDSMPRHISVRHTLQIISVKIKGIRGGLQWPINVFGLIAARDTIDRNRIMIFNRTRDNCQTITKEDRYLLLTGPTRAVVVSDPVYFEAPLKVKGSVESEDKDLSFLAVPLTGASDRGETRLVNREYTSRLSTLELTFGFVVESLEASISVRIIDGSWKDGFRGAFTAHTPSLKDNKVLLLDSGYCEMVPVTADRMIKLSRHVVSVEGEGDLTVSVLALGTDNVIEDEKDFTPKEAGMSQSSLDVGFCKLEVTVNWSLLSLLPDGYT*'

In [1379]:
x1 = str(Seq(str(df10.loc[160, 'Mutated']).replace('-', ''), generic_dna).reverse_complement().translate())
#str(Seq(str(df10.loc[121, 'Mutated']).replace('-', ''), generic_dna).translate())

In [1363]:
x2 = str(Seq(str(df10.loc[8, 'Extract']).replace('-', ''), generic_dna).reverse_complement().translate())

In [1364]:
for i in range(len(x1)):
    if x1[i] != x2[i]:
        print(i, x1[i], x2[i])

347 N I


In [566]:
str(Seq(str(df10.loc[69, 'Extract']).replace('-', ''), generic_dna).reverse_complement().translate()) == str(Seq(str(df10.loc[69, 'Mutated']).replace('-', ''), generic_dna).reverse_complement().translate())

False

In [1386]:
df10 = df10.drop(columns = ['Extract', 'Mutated'])

In [1388]:
pd.read_csv('../Figures and tables/All candidate genes.csv')

,Locus ID,QTL,annotation,Functionally annotated,Outlier in expression,Expression cluster,Special interest
0,LOC_Os01g65920,qCDP1,"F-box/LRR-repeat protein 2, putative, expressed",Yes,No,1,No
1,LOC_Os01g66000,qCDP1,"NADH dehydrogenase I subunit N, putative, expr...",No,No,1,No
2,LOC_Os01g66020,qCDP1,"protein kinase family protein, putative, expre...",No,No,2,No
3,LOC_Os01g66030,qCDP1,OsMADS2 - MADS-box family gene with MIKCc type...,No,No,1,No
4,LOC_Os01g66040,qCDP1,"receptor kinase, putative, expressed",No,No,1,No
...,...,...,...,...,...,...,...
335,LOC_Os12g41690,qCDP17,membrane associated DUF588 domain containing p...,No,No,1,No
336,LOC_Os12g41700,qCDP17,"LSD1 zinc finger domain containing protein, ex...",Yes,No,1,No
337,LOC_Os12g41710,qCDP17,"Protein kinase domain containing protein, expr...",No,No,2,No
338,LOC_Os12g41715,qCDP17,"DEAD-box ATP-dependent RNA helicase, putative,...",No,No,1,No


In [1396]:
df10 = pd.read_excel('../Figures and tables/340 substitution effects.xlsx')
df10 = pd.merge(df10, pd.read_csv('../Figures and tables/All candidate genes.csv').drop(columns = ['QTL'])
                , left_on = 'Gene', right_on = 'Locus ID')

In [1390]:
sum(df10['Gene'] == df10['Locus ID'])

28188

In [1397]:
df10.columns

Index(['#CHROM', 'POS', 'ID', 'Frameshift', 'Effect', 'Site', 'REF', 'ALT',
       'Gene', 'Strand', 'Start', 'End', 'QTL', 'Transcript', 'Gene POS',
       'MID', 'Locus ID', 'annotation', 'Functionally annotated',
       'Outlier in expression', 'Expression cluster', 'Special interest'],
      dtype='object')

In [1398]:
df10 = df10[['ID', '#CHROM', 'POS', 'REF', 'ALT', 'Site', 'Frameshift', 'Effect', 'QTL', 'Transcript', 'Gene POS', 'Expression cluster',
     'Functionally annotated', 'Outlier in expression', 'annotation']]

In [1400]:
df10.to_csv('../Figures and tables/SNP effects annotated.csv', index = False)

In [ ]:
pd.merge(pd.read_excel('../Figures and tables/340 substitution effects.xlsx'),
    pd.read_csv('../Figures and tables/SNP effects annotated.csv')[['Transcript', 
        'Expression cluster', 'Functionally annotated', 'Outlier in expression', 'annotation']]).drop_duplicates().to_csv('../Figures and tables/340 substitution effects annotated.csv', index = False)

In [3]:
df11 = pd.read_excel('../Figures and tables/340 substitution effects.xlsx')

In [4]:
df12 = pd.read_csv('../../WFC2.vcf', sep = '\t')

In [22]:
df11

,#CHROM,POS,ID,Frameshift,Effect,Site,REF,ALT,Gene,Strand,Start,End,QTL,Transcript,Gene POS,MID
0,chr01,38275059,1365463,no,synonymous,intron,A,G,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,4,0
1,chr01,38275062,1365464,no,synonymous,intron,G,GGTTACAGA,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,7,1
2,chr01,38275065,1365465,no,synonymous,intron,C,T,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,10,2
3,chr01,38275069,1365466,no,synonymous,intron,A,G,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,14,3
4,chr01,38275069,1365466,no,synonymous,intron,A,T,LOC_Os01g65920,reverse,38275055,38279030,qCDP1,LOC_Os01g65920.1,14,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28183,chr12,25840275,15180417,no,synonymous,exon,C,G,LOC_Os12g41720,reverse,25838564,25840479,qCDP17,LOC_Os12g41720.1,1711,28183
28184,chr12,25840309,15180418,no,missense,exon,G,C,LOC_Os12g41720,reverse,25838564,25840479,qCDP17,LOC_Os12g41720.1,1745,28184
28185,chr12,25840315,15180419,no,missense,exon,G,C,LOC_Os12g41720,reverse,25838564,25840479,qCDP17,LOC_Os12g41720.1,1751,28185
28186,chr12,25840379,15180420,no,missense,exon,C,A,LOC_Os12g41720,reverse,25838564,25840479,qCDP17,LOC_Os12g41720.1,1815,28186


In [5]:
df4 = pd.read_table('../Sequence and SNP data/Bout340can.tsv', header = None)
df4['Start'] = df4[[8,9]].min(axis = 1)
df4['End'] = df4[[8,9]].max(axis = 1)
t = []
for s,e in df4[[8,9]].itertuples(index = False):
    if s < e:
        t.append('forward')
    else:
        t.append('reverse')
df4['Strand'] = t

In [6]:
df5 = df12

In [7]:
t = []
for c, p in df5[['#CHROM', 'POS']].itertuples(index = False):
    f = False
    for l,o,s,e in df4[df4[1] == c][[0, 'Strand', 'Start', 'End']].itertuples(index = False):
        if s <= p <= e:
            t.append((l, o, s, e))
            f = True
            break
    if f == False:
        t.append(('None', o, s, e))

In [8]:
df5['Gene'] = list(zip(*t))[0]
df5['Strand'] = list(zip(*t))[1]
df5['Start'] = list(zip(*t))[2]
df5['End'] = list(zip(*t))[3]

In [9]:
df5 = df5[df5['Gene'] != 'None']

In [10]:
t = []
for i,r in df5.iterrows():
    if ',' in r['ALT']:
        for a in r['ALT'].split(','):
            r2 = r.copy()
            r2.loc['ALT'] = a
            t.append(r2)
    else:
        t.append(r)

In [11]:
df6 = pd.DataFrame(t)

In [12]:
for a in df6['ALT'].to_list():
    if ',' in a:
        print(a)

In [13]:
df6 = df6.applymap(lambda x: 0 if x == '0/0' else x).applymap(lambda x: str(x).count('/0') if '/0' in str(x) else x).applymap(lambda x: str(x).count('0/') if '0/' in str(x) else x).applymap(lambda x: 2 if '/' in str(x) else x)

In [14]:
df6

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,IRIS_313-10340,...,IRIS_313-9606,IRIS_313-9617,IRIS_313-9626,IRIS_313-9661,IRIS_313-9682,IRIS_313-9696,Gene,Strand,Start,End
0,chr01,38275059,1365463,A,G,.,PASS,.,GT,2,...,2,2,2,2,0,2,LOC_Os01g65920,reverse,38275055,38279030
1,chr01,38275062,1365464,G,GGTTACAGA,.,PASS,.,GT,0,...,0,0,0,0,0,0,LOC_Os01g65920,reverse,38275055,38279030
2,chr01,38275065,1365465,C,T,.,PASS,.,GT,2,...,2,2,2,2,0,2,LOC_Os01g65920,reverse,38275055,38279030
3,chr01,38275069,1365466,A,G,.,PASS,.,GT,2,...,2,2,2,2,0,2,LOC_Os01g65920,reverse,38275055,38279030
3,chr01,38275069,1365466,A,T,.,PASS,.,GT,2,...,2,2,2,2,0,2,LOC_Os01g65920,reverse,38275055,38279030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119596,chr12,25840275,15180417,C,G,.,PASS,.,GT,0,...,0,0,0,0,0,0,LOC_Os12g41720,reverse,25838564,25840479
119597,chr12,25840309,15180418,G,C,.,PASS,.,GT,2,...,2,2,1,2,2,2,LOC_Os12g41720,reverse,25838564,25840479
119598,chr12,25840315,15180419,G,C,.,PASS,.,GT,2,...,0,2,0,2,2,0,LOC_Os12g41720,reverse,25838564,25840479
119599,chr12,25840379,15180420,C,A,.,PASS,.,GT,0,...,0,0,0,0,0,0,LOC_Os12g41720,reverse,25838564,25840479


In [15]:
df7 = df6.drop(columns = ['#CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT', 'Gene', 'Strand', 'Start', 'End'])

In [47]:
df7 = pd.read_csv('../../WFC2L.vcf', sep = '\t')

In [50]:
df7 = df7.drop(columns = ['#CHROM', 'POS', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO', 'FORMAT'])

In [16]:
df8 = pd.read_table('../../../../Desktop/39ProcessedTraits.txt').transpose()

In [52]:
t = []
for i,r in df7.iterrows():
    for tr in df8.index:
        tdf = pd.concat([r[1:], df8.loc[tr, :]], axis = 1)
        g1 = tdf[tr][tdf[i] == 0].dropna()
        g2 = tdf[tr][tdf[i] == 1].dropna()
        g3 = tdf[tr][tdf[i] == 2].dropna()

        if min(len(g1), len(g2), len(g3)) < 4:
            continue

        x = stats.f_oneway(g1, g2, g3)

        t.append((r['MID'], len(g1), len(g2), len(g3), x.statistic, x.pvalue, tr))

In [53]:
df9 = pd.DataFrame(t, columns = ['MID', 'AA', 'Aa', 'aa', 'F statistic', 'p value', 'Trait'])

In [55]:
df9

,MID,AA,Aa,aa,F statistic,p value,Trait
0,33,20,4,147,1.301004,0.274987,Root_weight
1,33,20,4,147,1.049574,0.352371,Shoot_weight
2,33,20,4,147,2.544710,0.081518,Root_length
3,33,20,4,139,0.902027,0.407799,Filtered_Shoot_length
4,33,20,4,139,2.381646,0.095667,Filtered_Root_thickness
...,...,...,...,...,...,...,...
186095,28109,112,4,57,0.100022,0.904871,Height_WO_husk
186096,28109,112,4,57,0.851065,0.428771,Width_WO_husk
186097,28109,112,4,57,0.052342,0.949020,Seed_volume
186098,28109,106,4,57,0.469777,0.625980,Filtered_Seed_Weight


In [57]:
df10 = pd.read_csv('../Figures and tables/340 substitution effects annotated.csv')[['MID', 'Frameshift', 'Effect', 'Site', 
                                                                       'Transcript', 'ID', 'QTL', 'Expression cluster', 
                                                                       'Functionally annotated', 'Outlier in expression',
                                                               'annotation']]

In [58]:
df10 = pd.merge(df9, df10, how = 'left').sort_values('p value')

In [61]:
df10.drop_duplicates().to_csv('../Figures and tables/One-way ANOVA.csv', index = False)

In [63]:
t = []
for i,r in df7.iterrows():
    for tr in df8.index:
        tdf = pd.concat([r[1:], df8.loc[tr, :]], axis = 1)
        g1 = tdf[tr][tdf[i] == 0].dropna()
        g2 = tdf[tr][tdf[i] == 1].dropna()
        g3 = tdf[tr][tdf[i] == 2].dropna()

        if min(len(g1), len(g3)) < 5:
            continue

        x = stats.ttest_ind(g1, g3)

        t.append((r['MID'], len(g1), len(g3), x.statistic, x.pvalue, tr))

In [64]:
df11 = pd.DataFrame(t, columns = ['MID', 'AA', 'aa', 't statistic', 'p value', 'Trait'])

In [65]:
df12 = pd.read_csv('../Figures and tables/340 substitution effects annotated.csv')[['MID', 'Frameshift', 'Effect', 'Site', 
                                                                       'Transcript', 'ID', 'QTL', 'Expression cluster', 
                                                                       'Functionally annotated', 'Outlier in expression',
                                                               'annotation']]

In [66]:
df12 = pd.merge(df11, df12, how = 'left').sort_values('p value')

In [71]:
df12.drop_duplicates().to_csv('../Figures and tables/t-test.csv', index = False)

4

In [1001]:
df12[df12['p value'] < 0.001].groupby(['Locus ID', 'Effect']).mean()[:50]

ID          AA          aa  t statistic  \
Locus ID       Effect                                                          
LOC_Os01g65920 synonymous  1.365495e+06   54.973684  107.684211    -1.403479   
LOC_Os01g66000 synonymous  1.366304e+06   34.600000  135.400000     3.554805   
LOC_Os01g66030 synonymous  1.366612e+06  111.541667   51.500000     1.044584   
LOC_Os01g66040 synonymous  1.366660e+06  104.000000   59.500000     2.145782   
LOC_Os01g66060 missense    1.366865e+06  112.615385   47.846154     2.063799   
               synonymous  1.366850e+06  102.750000   59.416667     2.265041   
LOC_Os01g66110 missense    1.368122e+06  130.565217   25.565217     1.220734   
               synonymous  1.368097e+06  139.941558   22.844156     1.076566   
LOC_Os01g66120 missense    1.368647e+06  108.000000   60.000000     4.255400   
               synonymous  1.368643e+06  122.833333   41.166667    -0.763426   
LOC_Os01g66140 missense    1.368991e+06   20.500000  144.000000     0.421889   
               synonymous  1.369035e+06  108.590909   55.784091    -2.737116   
LOC_Os01g66160 synonymous  1.369505e+06   67.500000   96.250000     0.191171   
LOC_Os01g66240 missense    1.370540e+06  100.666667   63.166667     2.431589   
               synonymous  1.370550e+06   53.428571  115.428571     1.540369   
LOC_Os01g66250 missense    1.370588e+06  101.428571   61.785714     1.542496   
               synonymous  1.370604e+06   70.333333   91.333333     1.715246   
LOC_Os01g66280 missense    1.370785e+06   20.000000  151.000000     3.498992   
               synonymous  1.370781e+06  106.181818   56.000000     1.722907   
LOC_Os01g66290 missense    1.371087e+06  111.500000   48.500000     1.925003   
               synonymous  1.371069e+06  108.675000   50.550000     1.811471   
LOC_Os01g66350 synonymous  1.371829e+06  111.000000   46.900000     1.538115   
LOC_Os03g03350 synonymous  2.844229e+06   79.197802   88.868132     0.188524   
LOC_Os03g03360 synonymous  2.844260e+06  133.466667   33.900000    -2.037795   
LOC_Os03g03390 missense    2.844635e+06   24.500000  138.500000    -0.656572   
               synonymous  2.844627e+06  119.901961   45.019608     1.316407   
LOC_Os03g03420 missense    2.844973e+06   13.000000  155.000000     3.961728   
               synonymous  2.844962e+06   48.548387  118.209677     2.479963   
LOC_Os03g03430 missense    2.845382e+06  129.833333   33.500000    -3.842451   
               synonymous  2.845362e+06   50.750000  108.500000     1.722530   
LOC_Os03g03440 missense    2.845486e+06   24.533333  143.800000     3.314771   
               synonymous  2.845496e+06  109.925532   56.882979    -1.208332   
LOC_Os03g03450 missense    2.845671e+06   14.000000  158.000000     4.411988   
               synonymous  2.845597e+06  108.138889   57.500000    -0.256919   
LOC_Os03g03460 missense    2.845936e+06  157.000000   13.000000    -3.982707   
               synonymous  2.845945e+06  106.153846   59.564103    -0.295773   
LOC_Os03g03470 missense    2.846162e+06   14.000000  158.000000     4.407105   
               synonymous  2.846143e+06   79.000000   84.362069    -0.021265   
LOC_Os03g03500 synonymous  2.846619e+06  108.695652   48.260870     0.611763   
LOC_Os03g03510 synonymous  2.846693e+06   98.560000   65.840000    -0.299843   
LOC_Os03g03520 nonsense    2.846782e+06  154.500000   10.750000     1.826078   
               synonymous  2.846788e+06   45.187500  122.125000     1.279382   
LOC_Os03g03540 missense    2.846983e+06  153.000000   18.000000    -3.703007   
               synonymous  2.846941e+06   57.773810  104.964286     2.219430   
LOC_Os03g03550 missense    2.847378e+06   53.333333  113.000000     1.492504   
               synonymous  2.847386e+06   14.000000  156.214286     3.955943   
LOC_Os03g03560 missense    2.847405e+06   14.333333  155.666667     3.904733   
               synonymous  2.847402e+06   31.034483  138.586207     3.365504   
LOC_Os03g03570 missense    2.847473e+

In [1003]:
df12[(df12['Locus ID'] == 'LOC_Os03g03420')][:50]

,ID,AA,aa,t statistic,p value,Trait,Frameshift,Effect,Site,Locus ID,QTL,Expression cluster,Functionally annotated,Outlier in expression,annotation
25033,2844975,24,135,-5.419328,2.211851e-07,Filtered_Length_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein
24526,2844955,24,137,-5.407377,2.305911e-07,Filtered_Length_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein
24604,2844958,24,134,-5.354303,3.022623e-07,Filtered_Length_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein
24682,2844963,23,136,-5.102757,9.554724e-07,Filtered_Length_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein
24261,2844944,24,135,-5.072805,1.094113e-06,Filtered_Length_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein
24799,2844966,149,12,4.656973,6.734290e-06,Filtered_Length_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein
24917,2844970,158,14,-4.466920,1.442762e-05,Height_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein
24223,2844943,14,157,4.462549,1.474183e-05,Height_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein
24566,2844957,14,156,4.453637,1.535059e-05,Height_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein
24956,2844971,14,155,4.412971,1.823311e-05,Height_WO_husk,no,synonymous,intron,LOC_Os03g03420,qCDP3,2,No,No,expressed protein


In [711]:
df11 = pd.read_table('../../../../Desktop/tasoutall.txt')

In [722]:
df12 = pd.merge(df10,df11[['Trait', 'Marker', 'p', 'minorObs']], left_on = 'ID', right_on = 'Marker', how = 'right')

In [731]:
df12[df12['Effect'] != 'synonymous'].sort_values('p')[250:300]

,#CHROM,POS,ID,REF,ALT,Gene,Strand,Start,End,Locus_id,model,Hit key,Gene POS,Frameshift,Effect,Site,Trait,Marker,p,minorObs
314770,chr09,9491518.0,10903889.0,T,G,LOC_Os09g15510,reverse,9490521.0,9492544.0,LOC_Os09g15510,LOC_Os09g15510.1,LOC_Os09g15510_9490521,997.0,no,missense,exon,Shoot_weight,10903889,0.000010,37
316720,chr09,9497626.0,10904252.0,G,GTCC,LOC_Os09g15530,reverse,9497191.0,9498539.0,LOC_Os09g15530,LOC_Os09g15530.1,LOC_Os09g15530_9497191,435.0,no,missense,exon,Shoot_weight,10904252,0.000010,35
262986,chr08,27320706.0,10401147.0,C,T,LOC_Os08g43200,reverse,27320517.0,27321388.0,LOC_Os08g43200,LOC_Os08g43200.1,LOC_Os08g43200_27320517,189.0,no,missense,exon,Filtered_Stress_Shoot_length,10401147,0.000010,36
321790,chr09,9506511.0,10904725.0,A,C,LOC_Os09g15560,forward,9505694.0,9507077.0,LOC_Os09g15560,LOC_Os09g15560.1,LOC_Os09g15560_9505694,817.0,no,missense,exon,Shoot_weight,10904725,0.000010,37
318553,chr09,9499260.0,10904334.0,A,C,LOC_Os09g15540,forward,9498904.0,9500512.0,LOC_Os09g15540,LOC_Os09g15540.1,LOC_Os09g15540_9498904,356.0,no,missense,exon,Shoot_weight,10904334,0.000010,37
317461,chr09,9498197.0,10904272.0,C,A,LOC_Os09g15530,reverse,9497191.0,9498539.0,LOC_Os09g15530,LOC_Os09g15530.1,LOC_Os09g15530_9497191,1006.0,no,missense,exon,Shoot_weight,10904272,0.000010,36
317422,chr09,9498193.0,10904271.0,T,A,LOC_Os09g15530,reverse,9497191.0,9498539.0,LOC_Os09g15530,LOC_Os09g15530.1,LOC_Os09g15530_9497191,1002.0,no,missense,exon,Shoot_weight,10904271,0.000010,36
317500,chr09,9498200.0,10904273.0,T,A,LOC_Os09g15530,reverse,9497191.0,9498539.0,LOC_Os09g15530,LOC_Os09g15530.1,LOC_Os09g15530_9497191,1009.0,no,missense,exon,Shoot_weight,10904273,0.000010,36
322063,chr09,9506763.0,10904732.0,C,A,LOC_Os09g15560,forward,9505694.0,9507077.0,LOC_Os09g15560,LOC_Os09g15560.1,LOC_Os09g15560_9505694,1069.0,no,missense,exon,Shoot_weight,10904732,0.000010,36
323350,chr09,9511966.0,10905165.0,A,C,LOC_Os09g15570,reverse,9511314.0,9513125.0,LOC_Os09g15570,LOC_Os09g15570.1,LOC_Os09g15570_9511314,652.0,no,missense,exon,Shoot_weight,10905165,0.000010,36


In [735]:
df13 = pd.read_table('../../../../Desktop/tasoutaus.txt')
df14 = pd.read_table('../../../../Desktop/tasoutind.txt')

In [736]:
df13 = pd.merge(df10,df13[['Trait', 'Marker', 'p', 'minorObs']], left_on = 'ID', right_on = 'Marker', how = 'right')
df14 = pd.merge(df10,df14[['Trait', 'Marker', 'p', 'minorObs']], left_on = 'ID', right_on = 'Marker', how = 'right')

In [742]:
df14[df14['Effect'] != 'synonymous'].sort_values('p')[50:100]

,#CHROM,POS,ID,REF,ALT,Gene,Strand,Start,End,Locus_id,model,Hit key,Gene POS,Frameshift,Effect,Site,Trait,Marker,p,minorObs
270733,chr08,27376696.0,10403062.0,T,A,LOC_Os08g43320,reverse,27376015.0,27379035.0,LOC_Os08g43320,LOC_Os08g43320.1,LOC_Os08g43320_27376015,681.0,no,missense,exon,Height_WO_husk,10403062,0.000017,39
128578,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Height_WO_husk,7537763,0.000026,31
130801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Height_WO_husk,7538924,0.000027,37
128618,chr06,20783544.0,7537770.0,A,G,LOC_Os06g35620,reverse,20783542.0,20784024.0,LOC_Os06g35620,LOC_Os06g35620.1,LOC_Os06g35620_20783542,2.0,no,missense,exon,Width_WO_husk,7537770,0.000040,26
119179,chr06,20706869.0,7535187.0,CT,C,LOC_Os06g35520,reverse,20705349.0,20706997.0,LOC_Os06g35520,LOC_Os06g35520.1,LOC_Os06g35520_20705349,1520.0,yes,nonsense,exon,Height_WO_husk,7535187,0.000041,29
128547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Filtered_Shoot_length,7537763,0.000043,29
118556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Width_WO_husk,7535172,0.000048,27
120233,chr06,20719394.0,7535627.0,G,A,LOC_Os06g35530,forward,20717740.0,20724744.0,LOC_Os06g35530,LOC_Os06g35530.1,LOC_Os06g35530_20717740,1654.0,no,missense,exon,Width_WO_husk,7535627,0.000048,27
129008,chr06,20783917.0,7537779.0,C,G,LOC_Os06g35620,reverse,20783542.0,20784024.0,LOC_Os06g35620,LOC_Os06g35620.1,LOC_Os06g35620_20783542,375.0,no,missense,exon,Width_WO_husk,7537779,0.000048,27
129125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Width_WO_husk,7537807,0.000048,27


In [67]:
for t in df5['Trait'].unique():
    print(t)
    print(df5[df5['Trait'] == t].sort_values('p').iloc[0, 5])

Root_thickness
7.9383e-07
Stress_Root_Na+
0.00024827
Shoot_weight
5.9345e-07
Shoot_length
3.4648000000000003e-10
Stress_Shoot_Na+
4.2974e-06
Lost_Shoot_weight
0.00028683
Height_WO_husk
2.1116e-09
Root_length
5.7718e-07
Stress_Chlorophyll_A
1.3359999999999999e-08
Root_Na+
0.00057364
SES
3.7559e-06
Stress_Shoot_weight
4.1958000000000004e-08
Stress_Chlorophyll_B
3.1492e-09
Seed_density
0.00011375
Stress_Shoot_thickness
2.8386999999999996e-07
Seed_volume
4.5753000000000006e-08
Shoot_K+
5.893e-05
Seed_Weight
1.0843e-05
Stress_Root_thickness
1.3489e-05
Lost_Chlorophyll_A
8.209999999999999e-05
Stress_Root_length
2.1876e-06
Lost_Chlorophyll_B
3.9959e-05
Root_weight
3.1979e-05
Lost_Root_weight
0.00062454
Lost_Root_thickness
9.1598e-06
Chlorophyll_B
1.9577999999999998e-05
Lost_Shoot_thickness
0.00058148
Stress_Shoot_K+
2.1901e-05
Shoot_Na+
0.0002076
Stress_Root_weight
2.0039e-06
Chlorophyll_A
4.8755e-05
Stress_Shoot_length
2.7219000000000002e-11
Length_WO_husk
9.254200000000001e-13
Root_K+
0.000

In [70]:
for t in df6['Trait'].unique():
    print(t)
    print(df6[df6['Trait'] == t].sort_values('p').iloc[0, 5])

Root_thickness
0.00071101
Stress_Root_Na+
1.5345e-05
Shoot_weight
6.599399999999999e-05
Shoot_length
3.602e-05
Stress_Shoot_Na+
0.00041558
Lost_Shoot_weight
9.831e-05
Height_WO_husk
0.000352
Root_length
5.848600000000001e-07
Stress_Chlorophyll_A
3.8696e-05
Root_Na+
0.00021086
SES
1.2828e-05
Stress_Shoot_weight
9.4837e-06
Stress_Chlorophyll_B
9.7837e-05
Seed_density
0.00037697
Stress_Shoot_thickness
1.3813e-05
Seed_volume
4.428e-06
Shoot_K+
5.1566999999999996e-05
Seed_Weight
1.0005e-06
Stress_Root_thickness
6.3988e-05
Lost_Chlorophyll_A
0.0006111000000000001
Stress_Root_length
5.5438e-07
Lost_Chlorophyll_B
0.00029001
Root_weight
1.7254e-05
Lost_Root_weight
0.00068091
Lost_Root_thickness
0.00022241
Chlorophyll_B
7.3692e-05
Lost_Shoot_thickness
0.00036879
Stress_Shoot_K+
2.5770999999999998e-05
Shoot_Na+
0.00043545
Stress_Root_weight
6.594600000000001e-05
Chlorophyll_A
0.00099661
Stress_Shoot_length
2.0392e-05
Length_WO_husk
1.047e-05
Root_K+
0.00116
Shoot_thickness
2.7664e-05
Width_WO_hus

In [71]:
for t in df7['Trait'].unique():
    print(t)
    print(df7[df7['Trait'] == t].sort_values('p').iloc[0, 5])

Root_thickness
0.00158
Stress_Root_Na+
0.0018100000000000002
Shoot_weight
0.00017784
Shoot_length
0.00017126
Stress_Shoot_Na+
5.8009e-05
Lost_Shoot_weight
0.00089123
Height_WO_husk
2.0055999999999998e-08
Root_length
0.00037946
Stress_Chlorophyll_A
0.00157
Root_Na+
0.00049651
SES
4.705e-05
Stress_Shoot_weight
6.9748e-05
Stress_Chlorophyll_B
0.00057679
Seed_density
0.00372
Stress_Shoot_thickness
7.3176e-05
Seed_volume
1.6241e-05
Shoot_K+
0.00031667
Seed_Weight
4.0309e-05
Stress_Root_thickness
1.6713e-05
Lost_Chlorophyll_A
0.00088141
Stress_Root_length
0.00155
Lost_Chlorophyll_B
0.00039726
Root_weight
0.00015144
Lost_Root_weight
0.00013513
Lost_Root_thickness
0.0015
Chlorophyll_B
0.00021567
Lost_Shoot_thickness
0.00074557
Stress_Shoot_K+
5.7552e-05
Shoot_Na+
0.00022735
Stress_Root_weight
6.8835e-05
Chlorophyll_A
0.00010373
Stress_Shoot_length
1.2614e-06
Length_WO_husk
3.0251e-05
Root_K+
0.00083451
Shoot_thickness
0.00025984
Width_WO_husk
8.752e-09
Lost_Root_length
0.00017522
Stress_Root_K

In [73]:
df7.sort_values('p')[:50]

,Trait,Marker,Chr,Pos,marker_F,p,marker_Rsq,add_F,add_p,dom_F,dom_p,marker_df,marker_MS,error_df,error_MS,model_df,model_MS,minorObs
347773,Width_WO_husk,7533095,6,20635236,23.92036,8.752000e-09,0.38631,48.43947,9.902200e-10,0.01862,0.891830,2,15.17357,76,0.63434,2,15.17357,38
66003,Height_WO_husk,10403480,8,27385275,22.58373,2.005600e-08,0.37277,36.68990,4.749000e-08,6.06441,0.016060,2,13.76650,76,0.60958,2,13.76650,28
65980,Height_WO_husk,10403457,8,27384831,21.48583,4.018500e-08,0.36119,38.51848,2.530500e-08,3.30176,0.073150,2,13.33897,76,0.62083,2,13.33897,28
65993,Height_WO_husk,10403470,8,27384891,21.03615,5.361600e-08,0.35633,40.64498,1.233200e-08,1.27968,0.261510,2,13.15928,76,0.62556,2,13.15928,27
348276,Width_WO_husk,7539728,6,20832810,20.41341,8.022300e-08,0.34946,38.48595,2.558700e-08,1.89402,0.172790,2,13.72639,76,0.67242,2,13.72639,36
65858,Height_WO_husk,10403320,8,27381829,20.38452,8.174600e-08,0.34914,31.91820,2.587900e-07,6.55017,0.012470,2,12.89397,76,0.63254,2,12.89397,31
65900,Height_WO_husk,10403368,8,27382177,20.38452,8.174600e-08,0.34914,31.91820,2.587900e-07,6.55017,0.012470,2,12.89397,76,0.63254,2,12.89397,31
65921,Height_WO_husk,10403391,8,27383220,19.58497,1.380500e-07,0.34011,29.15840,7.162100e-07,7.53635,0.007540,2,12.56023,76,0.64132,2,12.56023,25
65853,Height_WO_husk,10403315,8,27381695,19.48863,1.471200e-07,0.33900,30.85932,3.811500e-07,6.08144,0.015910,2,12.51939,76,0.64239,2,12.51939,30
348183,Width_WO_husk,7539292,6,20822947,19.25619,1.716000e-07,0.33632,22.10596,1.111000e-05,12.96995,0.000562,2,13.20996,76,0.68601,2,13.20996,21


In [4]:
df = pd.read_excel('../Figures and tables/Candidate genes.xlsx')

In [7]:
df2 = pd.read_csv('../Figures and tables/All candidate genes.csv')

In [12]:
pd.merge(df, df2[['Locus ID', 'QTL', 'Expression cluster']], on = 'Locus ID', how = 'left').to_csv('../Figures and tables/Table 3 Reviewed candidate genes.csv', index = False)

In [18]:
df3 = pd.read_table('../../Data/MSU7/all.locus_brief_info.7.0')[['locus', 'annotation']]

In [19]:
df3

,locus,annotation
0,LOC_Os01g01010,"TBC domain containing protein, expressed"
1,LOC_Os01g01010,"TBC domain containing protein, expressed"
2,LOC_Os01g01019,expressed protein
3,LOC_Os01g01030,"monocopper oxidase, putative, expressed"
4,LOC_Os01g01040,expressed protein
...,...,...
66333,ChrSy.fgenesh.gene.85,expressed protein
66334,ChrSy.fgenesh.gene.86,expressed protein
66335,ChrSy.fgenesh.gene.87,expressed protein
66336,ChrSy.fgenesh.gene.88,hypothetical protein


In [27]:
df3 = pd.merge(pd.read_csv('../Figures and tables/Table 3 Reviewed candidate genes.csv'), 
         df3.rename(columns = {'locus' : 'Locus ID'}), how = 'left').drop_duplicates()

In [31]:
pd.read_table('../../Data/raptomsu.txt').drop_duplicates

,RAP,MSU7,Affymetrix probe ID
0,Locus_ID,No hit,No hit
1,Os01g0100100,LOC_Os01g01010.1,Os.24643.1.S1_at
2,Os01g0100100,LOC_Os01g01010.2,Os.24643.1.S1_at
3,Os01g0100200,LOC_Os01g01019.1,Os.37240.1.S1_at
4,Os01g0100300,None,No hit
...,...,...,...
82114,Os12g0641400,LOC_Os12g44380.3,Os.26353.1.S1_at
82115,Os12g0641500,LOC_Os12g44390.1,Os.8335.2.S1_at
82116,Os12g0641500,LOC_Os12g44390.1,Os.8335.2.S1_at
82117,Os12g0641500,LOC_Os12g44390.1,Os.8335.2.S1_at
